# Measuring and Predicting Student Dropout in Yiya Air Science Courses

In this project, our first aim to identify the points in the Yiya Air Science courses 2 and 3 where students dropout. There are different ways extents and ways to measure drop out. For example, dropout can be defined as a:
1. **Usage Gap**, where the time since the students last response is more than a certain amount of days e.g (e.g more than 7 days since last response). This can be seen as students being "on track" or "behind".
2. **Total Response Stop**, where a student has sent their last response to the system before completing all tasks
3. **Total Question Stop**, where a student stops submitting graded problems in the course before completing all tasks [(Borella, et al., 2019)](https://mitili.mit.edu/sites/default/files/project-documents/a24-Borrella_Caballero_Ponce_2019.pdf)
4. **Total Completion Stop**, where a student stops completing tasks (lesson modules and tests)
5. **Opt out** - student voluntarily unenrolls from the course without achieving a certificate [(Tinto, 1975)](https://www-tandfonline-com.cmu.idm.oclc.org/doi/full/10.1080/01587919.2017.1369006#)

Our second aim is to measure and visualize the rate of each of these dropout metrics among students over time and each course step. We can do so by diving the total occurances by the number of studnets. (Counts of students meeting dropout criteria / total number of students).

Our third aim is to identify features (variables) that are predictive of dropout with a classification model. Prior research suggests a need to refine success and dropout metrics [(Kohler, 2019)](https://www-tandfonline-com.cmu.idm.oclc.org/doi/full/10.1080/01587919.2017.1369006). It is uncertain if we have all of the following features in our data. However, we can try features grounded in previous theories, empircal results, and our best judgement on what makes sense for the context of the learning environment and the student population.  but some features that we may be able to use in our model and evaluate for predictive power include: 
- Student's date of first submission of a problem [(Liyanagunawardena, 2014)](https://centaur.reading.ac.uk/36002/)
- Student Intentions [(Liyanagunawardena, 2014)](https://centaur.reading.ac.uk/36002/)
    - Unsure
    - Browse Course
    - Audit
    - Complete
- Participation type [(Kohler, 2013)](https://er.educause.edu/articles/2013/6/retention-and-intention-in-massive-open-online-courses-in-depth)
    - Browser
    - Passive Participant 
    - Active Participant
    - Community Contributor
We can try to align participant types with [Yiya User Definitions](https://docs.google.com/document/d/1KNgPFdRSBniQouaKIQQPuI-DlAL-2xs70t4rpps5Qq4/edit)
- Completion Behavior
    - Number of lesson modules (tasks) completed
    - Number of lesson questions completed
    - Number of questions answered correctly
    - Number of tests they've completed 
    - Whether they are a returning learner - There should be a record for each Registration, Previously registered key in response as well. Data team will have to get back to us on how to best query the data to get an accurate user.
- Time on Task metric
    - Minutes spent on a task / Average time of session - We can use the telcomsession table and the created and duration feature. There can be telecom issues so users can get kicked out before completing the script. We'll have to consider this. We may have to compare with the tasks table.
- Review Behaviors 
    - Number of Times Revising Previous Lesson
- Effect of payment
    - Payment Status (Scholarship or Self Paid with mobile money) 
    - payment survey question - Required question in the response table. Shariffa will share the key. 
    
Note about "effect of payment": The date at which non-paid users lost access to course 3 was [*To be confirmed by yiya team, date was potentially nov 4th*]
Some required questions have required in their key?